In [1]:
## prevent underfitting, use a model enough to complicate
## -> prevent overfitting, use a Lasso model (L1 regularization)

## import modules
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

from math import sqrt

import pandas as pd
import numpy as np

In [2]:
admission_df = pd.read_csv('data/admission_data.csv').drop('Serial No.', axis=1)
admission_df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1,0.87
496,337,117,5,5.0,5.0,9.87,1,0.96
497,330,120,5,4.5,5.0,9.56,1,0.93
498,312,103,4,4.0,5.0,8.43,0,0.73


In [3]:
X = admission_df.loc[:,'GRE Score':'Research']
X.values

array([[337.  , 118.  ,   4.  , ...,   4.5 ,   9.65,   1.  ],
       [324.  , 107.  ,   4.  , ...,   4.5 ,   8.87,   1.  ],
       [316.  , 104.  ,   3.  , ...,   3.5 ,   8.  ,   1.  ],
       ...,
       [330.  , 120.  ,   5.  , ...,   5.  ,   9.56,   1.  ],
       [312.  , 103.  ,   4.  , ...,   5.  ,   8.43,   0.  ],
       [327.  , 113.  ,   4.  , ...,   4.5 ,   9.04,   0.  ]])

In [4]:
## make 6th degree polynomial regression model
polynomial_transformer = PolynomialFeatures(6)
polynomial_features = polynomial_transformer.fit_transform(X.values)
features = polynomial_transformer.get_feature_names(X.columns)

In [5]:
X = pd.DataFrame(polynomial_features, columns=features)
X.shape

(500, 1716)

In [6]:
y = admission_df[['Chance of Admit ']]
y

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65
...,...
495,0.87
496,0.96
497,0.93
498,0.73


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [8]:
#model = LinearRegression()
model = Lasso(alpha=0.001, max_iter=1000, normalize=True)
## alpha : coefficient of regularization term (lambda)
## max_iter : how many gradient descent steps?
## normalize : feature scaling
model.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [9]:
y_train_predict = model.predict(X_train)

In [10]:
y_test_predict = model.predict(X_test)

In [11]:
mse = mean_squared_error(y_train, y_train_predict)

print("Predict power in training set")
print("-----------------------------")
print(sqrt(mse))
print()

mse = mean_squared_error(y_test, y_test_predict)

print("Predict power in test set")
print("-----------------------------")
print(sqrt(mse))

Predict power in training set
-----------------------------
0.06336620966147144

Predict power in test set
-----------------------------
0.06007719092689256


In [12]:
## practice with another dataset

In [13]:
## read dataset
insurance_df = pd.read_csv('data/insurance.csv')
insurance_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [17]:
## one-hot encoding for categorical data
insurance_df = pd.get_dummies(insurance_df, columns=['sex', 'smoker', 'region'])

In [18]:
X = insurance_df.drop(['charges'], axis=1)
X

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,0,1,1,0,0,1,0,0
1334,18,31.920,0,1,0,1,0,1,0,0,0
1335,18,36.850,0,1,0,1,0,0,0,1,0
1336,21,25.800,0,1,0,1,0,0,0,0,1


In [19]:
polynomial_transformer = PolynomialFeatures(4)
polynomial_features = polynomial_transformer.fit_transform(X.values)
features = polynomial_transformer.get_feature_names(X.columns)

In [21]:
polynomial_features.shape

(1338, 1365)

In [26]:
len(features)

1365

In [28]:
## training, test set
X = pd.DataFrame(polynomial_features, columns=features)
y = insurance_df[['charges']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [29]:
## make Lasso model
model = Lasso(alpha=1, max_iter=2000, normalize=True)
model.fit(X_train, y_train)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=2000, normalize=True,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [30]:
## test model's performance
y_train_predict = model.predict(X_train)
mse = mean_squared_error(y_train, y_train_predict)

print("Performance at training set")
print("---------------------------")
print(f'RMSE: {sqrt(mse)}')
print()

y_test_predict = model.predict(X_test)
mse = mean_squared_error(y_test, y_test_predict)

print("Performance at test set")
print("---------------------------")
print(f'RMSE: {sqrt(mse)}')

Performance at training set
---------------------------
RMSE: 4726.636439607449

Performance at test set
---------------------------
RMSE: 4692.232442526968
